In [57]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [58]:
# カレントスレッドにデフォルトのグラフが残存していることがあるので、リセットする
tf.reset_default_graph()

In [59]:
sess = tf.InteractiveSession()

In [60]:
# 再現性の確保のために乱数シードを固定(数値は何でもよい)
tf.set_random_seed(12345)

In [61]:
# 入力データ
# MNISTのone-hot表現での読み込み
mnist = input_data.read_data_sets("MNIST_data2/", one_hot=True)

Extracting MNIST_data2/train-images-idx3-ubyte.gz
Extracting MNIST_data2/train-labels-idx1-ubyte.gz
Extracting MNIST_data2/t10k-images-idx3-ubyte.gz
Extracting MNIST_data2/t10k-labels-idx1-ubyte.gz


In [62]:
# 入力画像
# プレースホルダー型で・・・
x = tf.placeholder(tf.float32, name='x')

In [63]:
# サイズ変更
x_1 = tf.reshape(x, [-1, 28, 28, 1])

In [64]:
# ランダムカーネル
k_0 = tf.Variable(tf.truncated_normal([4, 4, 1, 10], mean=0.0, stddev=0.1))

# 畳み込み
x_2 = tf.nn.conv2d(x_1, k_0, strides=[1, 3, 3, 1], padding='VALID')

In [65]:
# 活性化関数
x_3 = tf.nn.relu(x_2)

In [66]:
# プーリング
x_4 = tf.nn.max_pool(x_3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

In [67]:
# サイズ変更
x_5 = tf.reshape(x_4, [-1, 160])

In [68]:
# ドロップアウト付きの全結合
# tf.nn.dropout()の引数のkeep_propにノードを残す割合を指定する
def matmul_plus_bias_with_dropout(x, w, b, p):
    return tf.matmul(tf.nn.dropout(x, keep_prob=p), w) + b

In [69]:
# 重みとバイアス
w_1 = tf.Variable(tf.zeros([160, 40]))
b_1 = tf.Variable([0.1] * 40)

# ドロップアウト率
p_1 = tf.placeholder(1.0, name='p_1')

# 全結合
# x_6 = tf.matmul(x_5, w_1) + b_1
x_6 = matmul_plus_bias_with_dropout(x_5, w_1, b_1, p_1)

In [70]:
# 活性化関数
x_7 = tf.nn.relu(x_6)

In [71]:
# 重みとバイアス
w_2 = tf.Variable(tf.zeros([40, 10]))
b_2 = tf.Variable([0.1] * 10)

# ドロップアウト率
p_2 = tf.placeholder(1.0, name='p_2')

# 全結合
# x_8 = tf.matmul(x_7, w_2) + b_2
x_8 = matmul_plus_bias_with_dropout(x_7, w_2, b_2, p_2)

In [72]:
# 確率化
y = tf.nn.softmax(x_8)

In [73]:
# 損失関数の最小化

# 正解ラベル
labels = tf.placeholder(tf.float32, name='labels')

# 損失関数(交差エントロピー誤差)
# tf.reduce_sum() : 与えたリストに入っている数値の合計を求めるメソッド
loss = -tf.reduce_sum(labels * tf.log(y))

# 最適化処理(Adam)
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [74]:
# 精度検証

# tf.equal(x, y) : xとyが等しければTrue、等しくなければFalseを返す
prediction_match = tf.equal(tf.argmax(y, axis=1), tf.argmax(labels, axis=1))

# tf.reduce_mean() : 与えたリストに入っている数値の平均値を求めるメソッド
# tf.cast() : 型を変換するメソッド　以下では第1引数がTrueなら、
# それを第2引数のfloat型に変換する
accuracy = tf.reduce_mean(tf.cast(prediction_match, tf.float32), name = "accuracy")

In [75]:
# パラメータ

# バッチサイズ
BATCH_SIZE = 32  
 
# 学習回数
NUM_TRAIN = 10_100  # 10_100 は 10,100 と同義

# 学習中の出力精度
OUTPUT_BY = 500

# ドロップアウト率
DROPOUT_PROB_1 = 0.2
DROPOUT_PROB_2 = 0.5

In [76]:
# 学習の実行
sess.run(tf.global_variables_initializer())
dropout_prob = {p_1 : DROPOUT_PROB_1, p_2 : DROPOUT_PROB_2}

# TensorFlowでは、モデルのファイル入出力は tf.train.Saver() を使う
saver = tf.train.Saver()

for i in range(NUM_TRAIN):
    # MNISTの訓練データから複数の画像データとラベルデータを取得(ミニバッチ学習)
    batch = mnist.train.next_batch(BATCH_SIZE)
    inout = {x : batch[0], labels : batch[1]}
    if i % OUTPUT_BY == 0:
        # train_accuracy = accuracy.eval(feed_dict=inout)
        train_accuracy = accuracy.eval(feed_dict = {**inout, p_1 : 1.0, p_2 : 1.0})
        print('step{:d}, accuracy{:.2f}'.format(i, train_accuracy))

        # 過程の保存
        saver.save(sess, 'models/my-model', global_step=i)

    # optimizer.run(feed_dict=inout)
    optimizer.run(feed_dict={**inout, **dropout_prob})

# 最終結果の保存
saver.save(sess, 'models/my-model')

"""
modelsの中に .data-00000-of-00001/.index/.meta という3つの拡張子が
付いたファイル群とcheckpointファイルが作成される
「過程の保存」では、ファイル名には「<モデル名>-<ステップ数>.<拡張子>」という名前で保存。
「最終結果の保存」では、ファイル名には「<モデル名>.<拡張子>」という名前で保存。

.metaファイルは、メタグラフというデータフローグラフ構造などの情報を保存したもの。
"""

step0, accuracy0.06
step500, accuracy0.62
step1000, accuracy0.59
step1500, accuracy0.78
step2000, accuracy0.66
step2500, accuracy0.78
step3000, accuracy0.81
step3500, accuracy0.78
step4000, accuracy0.94
step4500, accuracy0.81
step5000, accuracy0.81
step5500, accuracy0.72
step6000, accuracy0.84
step6500, accuracy0.94
step7000, accuracy0.78
step7500, accuracy0.94
step8000, accuracy0.91
step8500, accuracy0.84
step9000, accuracy0.78
step9500, accuracy0.84
step10000, accuracy0.84


'models/my-model'

In [77]:
# テストデータによる精度検証
# test_accuracy = accuracy.eval(feed_dict={x : mnist.test.images,\
#   labels : mnist.test.labels})
test_accuracy = accuracy.eval(feed_dict={x : mnist.test.images,\
    labels : mnist.test.labels, p_1 : 1.0, p_2 :1.0})
print('test_accuracy{:.2f}'.format(test_accuracy))

test_accuracy0.87
